In [1]:
!pip install numpy -q
!pip install pandas -q
!pip install matplotlib -q
!pip install tensorflow -q

!pip install opendatasets -q

ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


# **Importing Libraries**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import opendatasets as od

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
od.download("https://www.kaggle.com/datasets/tongpython/cat-and-dog")

In [ ]:
Batch_size = 32
Image_size = (128,128)

In [ ]:
train_data_dir = "/content/cat-and-dog/training_set/training_set"
test_data_dir = "/content/cat-and-dog/test_set/test_set"

In [ ]:
train_data = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                       image_size = Image_size,
                                                       batch_size = Batch_size,
                                                       subset = 'training',
                                                       validation_split = 0.1,
                                                       seed = 42)

validation_data = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                       image_size = Image_size,
                                                       batch_size = Batch_size,
                                                       subset = 'validation',
                                                       validation_split = 0.1,
                                                       seed = 42)

In [ ]:
test_data = tf.keras.utils.image_dataset_from_directory(test_data_dir,
                                                       image_size = Image_size,
                                                       batch_size = Batch_size)

# **Dataset Description**

In [ ]:
class_names = train_data.class_names
class_names

In [ ]:
for image_batch,label_batch in train_data.take(1):
  print(image_batch.shape)
  print(label_batch.shape)

In [ ]:
plt.figure(figsize=(10,4))
for image,label in train_data.take(1):
  for i in range(12):
    ax = plt.subplot(2,6,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(class_names[label[i]])
    plt.axis("off")

# **Data Preprocessing**

In [ ]:
for image,label in train_data.take(1):
  for i in range(1):
    print(image)

In [ ]:
train_data = train_data.map(lambda x,y : (x/255,y))
validation_data = validation_data.map(lambda x,y : (x/255,y))
test_data = test_data.map(lambda x,y : (x/255,y))

In [ ]:
for image,label in train_data.take(1):
  for i in range(1):
    print(image)

In [ ]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical", input_shape=(128, 128, 3)),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
])


In [ ]:
model = tf.keras.Sequential()

model.add(data_augmentation)

# Convolutional layers
model.add(tf.keras.layers.Conv2D(64, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(128, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(256, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Dropout(0.3))

# Flatten and dense layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))

model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data,
                    epochs=50,
                    validation_data=validation_data)

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend()
plt.show()


In [ ]:
fig = plt.figure()
plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend()
plt.show()

# **Model Evaluation**

In [ ]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
accuracy = tf.keras.metrics.BinaryAccuracy()

In [ ]:
for batch in test_data.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    accuracy.update_state(y, yhat)

In [ ]:
precision.result().numpy(), recall.result().numpy(), accuracy.result().numpy()

In [ ]:
!pip install opencv-python

In [ ]:
import cv2

In [ ]:
image = cv2.imread('/content/cat-and-dog/test_set/test_set/cats/cat.4330.jpg')
plt.imshow(image)
plt.show()

In [ ]:
resized_image = tf.image.resize(image, Image_size)
scaled_image = resized_image/255

In [ ]:
scaled_image

In [ ]:
np.expand_dims(scaled_image,0).shape

In [ ]:
y_hat = model.predict(np.expand_dims(scaled_image,0))

In [ ]:
y_hat

In [ ]:
class_names

In [ ]:
if y_hat > 0.5:
    print(f'Predicted class is Dog')
else:
    print(f'Predicted class is Cat')